In [2]:
import warnings
warnings.filterwarnings("ignore")

from utils import dataset, save_bow
import pandas as pd
import numpy as np

#gensim
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import Phrases, phrases

#spacy
import spacy

#tqdm
from tqdm import tqdm

In [4]:
corpus_name = '25_venues'
df = dataset()
df = df.iloc[np.where(~df['Abstract'].isna())]

In [5]:
len(df)

164756

In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    lemmatized = list(tqdm(nlp.pipe(texts, batch_size=1000, n_process=-1), total=len(texts)))
    lemmatized = [" ".join([token.lemma_ for token in text if token.pos_ in allowed_postags]) for text in lemmatized]
    return lemmatized

df['lemmatized']  = lemmatization(df['Abstract'])

100%|██████████| 164756/164756 [41:57<00:00, 65.45it/s]  


In [7]:
# Pre-Process
df['lemmatized'] = df['lemmatized'].apply(lambda x: simple_preprocess(x, deacc=True))

#Bigrams
bigram_phrases = Phrases(df['lemmatized'], min_count=5, threshold=50)
bigram = phrases.Phraser(bigram_phrases)
df['lemmatized'] = df['lemmatized'].apply(lambda x: bigram[x])

In [15]:
# save the df
df.to_csv(f'./database/{corpus_name}_lemmatized.csv', index=False)
save_bow(df['lemmatized'].tolist(), corpus_name)